Please find the documentation here:

Dense layer: https://keras.io/api/layers/core_layers/dense/

Sequential model: https://keras.io/guides/sequential_model/

to_categorical: https://keras.io/api/utils/python_utils/#to_categorical-function

activation: https://keras.io/api/layers/activations/

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
import numpy as np

from os import listdir
from os.path import isfile, join, isdir



In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow as tf
from tensorflow.keras import activations
from tensorflow.keras import utils

# Transform your data into matrices

## 1. Define the classes 

First we want to define the classes into which we want to classify the data. In the folder 'bbc' you may find 1 folder per category. Each folder holds a list of txt files, all representing one text that belongs to that specific category.

Can you find a way to read the different categories automatically from the 'bbc' folder

In [47]:
path_origin = 'bbc/'

categories = 

#Don't change this.
print(categories)
assert sorted(categories) == ['business', 'entertainment', 'politics', 'sport', 'tech']

['entertainment', 'business', 'sport', 'politics', 'tech']


## 2. Read the data into x and y

Construct two lists x and y. 
- x has to be a list of strings where 1 string is the data within 1 .txt file and we want to read all the subfolders within /bbc/.
- y has the same length of x but holds the corresponding label that belongs to the text in x. So the text on position 50 in x has the label on position 50 in y.

Tip: sometimes their may be errors when trying to read a file. You don't have to focus on solving this problem. Just use try: ... except: to skip the files that pose problems. Be sure to only add a label to y when you have actually managed to read a file and add it to X.

In [48]:
X = 
y = 


#Don't change this.            
print(len(X), len(y))
assert len(X) == len(y)

bbc/entertainment/
bbc/business/
bbc/sport/
bbc/politics/
bbc/tech/
2224 2224


## 3. Split you data

We need to make a seperate test and training dataset. We use the train set to train a model. Then we use the test set to check how the model performs on data it has not seen before.

For this you can use the train_test_split() of sklearn. Please look up how it works. We want to have 25% of the data of X in the test set.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(...)

[386, 510, 510, 417, 401]
[283, 372, 399, 318, 296]
[103, 138, 111, 99, 105]


Now this is done, we want to make sure all categories are well present in both datasets. Please run the cell below and check how much of each category we have within each dataset. Do you think all categories are enough represented?

In [ ]:
#You don't have to change this, you just have to run it.
print([y.count(cat) for cat in categories])
print([y_train.count(cat) for cat in categories])
print([y_test.count(cat) for cat in categories])

## 4. TF-IDF

As a next step we want to transform our data into a matrix, using TF-IDF. We will use the predefined function of sklearn TFIDFVectorizer. Please have a look at the different arguments you can give. Are their any interesting arguments that might be interesting to add? Please make sure you leave max_features= num_features.

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [7]:
#Choose the number of features you want to learn, 
#this is equal to the number of words you want to take into account in your vocabulary.
num_features = 10000
#Number of categories. We will need this later.
num_cat = len(categories)

#Play with this
tfidf = TfidfVectorizer(max_features= num_features, ...)

#Don't change this, but understand what is happening.
def tfidf_features(txt, flag):
    if flag == "train":
        x = tfidf.fit_transform(txt)
    else:
        x = tfidf.transform(txt)
    return x 


X_trans_train = tfidf_features(X_train, flag="train")
X_trans_test = tfidf_features(X_test, flag="test")

print(X_trans_train.shape, X_trans_test.shape)

(1668, 10000) (556, 10000)


## 5. Transform y into a matrix

Now the y vector is also made out of string, but remember, computers don't understand words. We have to attach a number to each class and perform a 1-hot encoding. See image below to understand 1-hot encoding or read more on the following link:

https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/#:~:text=A%20one%20hot%20encoding%20is,is%20marked%20with%20a%201.

<img src="assets/1hot.png" width=1000 height=1000 />

In [19]:
#You don't have to change any code in this cell. Please understand what is happening.
#First we transform the text label to a number
lb = LabelEncoder()
lb.fit(categories)
y_trans_train = lb.transform(y_train)
y_trans_test = lb.transform(y_test)

#Then we transform this to a one hot encoded matrix
y_hot_train=utils.to_categorical(y_trans_train, num_cat)
y_hot_test=utils.to_categorical(y_trans_test, num_cat)

print(y_trans_train.shape, y_trans_test.shape)

(1668,) (556,)


# Define your model

We are ready preparing our input and output model. Now we have to indicate how we want to build the model. Please fill in the gaps + add as much hidden layers as you see necessary

In [ ]:
model = tf.keras.Sequential()
#define input layer
model.add(layers.Dense(..., input_shape=(num_features,)))
model.add(layers.Activation(activations.relu))

#Add hidden layers

model.add(layers.Dense(num_cat))
model.add(layers.Activation(activations.softmax))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print (model.summary())

Now that we have defined the model, we will train it by feeding it with the input and output matrices we have prepared

In [59]:
# Model Training
nb_epochs = 1
batch_size = 64

history = model.fit(X_trans_train, y_hot_train, batch_size=batch_size, epochs=nb_epochs,verbose=1)

Epoch 1/1
1668/1668 [==============================] - 4s 2ms/step - loss: 1.3966 - accuracy: 0.5342


Now let the model make predictions based on
1. the input data it has already seen before
2. the new test data

In [15]:
y_train_predclass = model.predict_classes(X_trans_train,batch_size=batch_size)
y_test_predclass = model.predict_classes(X_trans_test,batch_size=batch_size)

Compare the performance of the two datasets

In [ ]:
#Model Prediction

print("nnDeep Neural Network - Train accuracy:",round(accuracy_score( y_trans_train, y_train_predclass),3))

print("nDeep Neural Network - Test accuracy:",round(accuracy_score( y_trans_test,y_test_predclass),3))

print("nDeep Neural Network - Train Classification Report")

print (classification_report(y_trans_train,y_train_predclass))

print("nDeep Neural Network - Test Classification Report")

print (classification_report(y_trans_test,y_test_predclass))